In [ ]:
pip install pyscheduling

In [1]:
from pyscheduling.Problem import Branch_Bound
from pyscheduling.FS import FmCmax, FlowShop
import threading
import pickle
import numpy as np
from time import perf_counter
import time
import matplotlib.pyplot as plt
import random
import pyscheduling as ps

## Classe branch&bound pour flow job permutation

In [67]:
class FS_Branch_Bound(Branch_Bound):
    choice="last_machine-makespan"
    M=float('inf')
    def branch(self, node: Branch_Bound.Node):
        if node.partial_solution is None :
            node.partial_solution=[]
        node.sub_nodes = []
        for i in range(instance.n):
            if(i not in node.partial_solution):
                sub_node=Branch_Bound.Node()
                sub_node.partial_solution=node.partial_solution.copy()
                sub_node.partial_solution.append(i)
                node.sub_nodes.append(sub_node)
                if(len(node.partial_solution)==instance.n-1):
                    sub_node.if_solution=True
                    
    def bound(self, node: Branch_Bound.Node):
        
        """bounding method
        Args:
            node (Node): node to bound
        """
        if(self.choice=="last_machine-makespan"):
            node.lower_bound = last_machine_makespan(self.instance ,node.partial_solution)
        elif(self.choice=="minseq_bound"):
            node.lower_bound = minseq_bound(self.instance, node.partial_solution)
        elif(self.choice=="lb2_bound"):
            node.lower_bound = lb2_bound(self.instance, node.partial_solution)
        elif(self.choice=="taillard"):
            node.lower_bound=taillard(node.partial_solution,self.instance)
        else:
            node.lower_bound = para_bound(self.instance, node.partial_solution)
        

 
    def objective(self, node: Branch_Bound.Node):
        solution = FlowShop.FlowShopSolution(self.instance,job_schedule=[ps.Problem.Job(job_id,0,0) for job_id in node.partial_solution])
        solution.compute_objective()
        node.lower_bound = solution.objective_value
        return solution.objective_value
        
    def solution_format(self, partial_solution: object, objective_value):
        solution = FlowShop.FlowShopSolution(self.instance, job_schedule=[ps.Problem.Job(job_id,0,0) for job_id in partial_solution])
        solution.compute_objective()
        return solution
    def solve(self, root : Branch_Bound.Node = None):
        global stop_flag
        if stop_flag:
            return
        if time.time() - start_time > 600:
            stop_flag = True
            return

        """recursive function to perform Branch&Bound on the instance attribute
        Args:
            root (Node, optional): starting node. Defaults to None.
        """
        if root is None:
            root = self.Node()
            self.root = root
            self.start_time = perf_counter()
        self.branch(root) 
        if root.sub_nodes[0].if_solution is False :
            for node in root.sub_nodes: self.bound(node)
            sorted_sub_nodes = root.sub_nodes
            sorted_sub_nodes.sort(reverse= self.instance.get_objective().value > 0, key = lambda node : node.lower_bound)
            for node in sorted_sub_nodes :
                if self.best_solution is not None : 
                    if self.instance.get_objective().value > 0 and node.lower_bound < self.objective_value : node = None
                    elif self.instance.get_objective().value < 0 and node.lower_bound > self.objective_value : node = None
                if node is not None : self.solve(node)
        else :
            for node in root.sub_nodes: 
                node.lower_bound = self.objective(node)
                solution = self.solution_format(node.partial_solution,node.lower_bound)
                self.all_solution.append(solution)
                if self.best_solution is None or (self.instance.get_objective().value > 0 and self.objective_value < node.lower_bound) :
                    self.objective_value = node.lower_bound
                    self.best_solution = solution
                elif self.best_solution is None or (self.instance.get_objective().value < 0 and (node.lower_bound < self.objective_value and node.lower_bound<M ) ) :
                    self.objective_value = node.lower_bound
                    self.best_solution = solution
        self.runtime = perf_counter() - self.start_time

## Lower bound methods

### 1- minseq_bound

In [3]:
def minseq_bound(instance, partial_solution):
    n, m = instance.P.shape
    k = len(partial_solution)
    rest = np.delete(instance.P, partial_solution, axis=0)
    if np.size(rest) != 0:
        min_vals = np.min(rest, axis=0)
        new_rest = np.tile(min_vals, (n-k, 1))
    else:
        new_rest = rest
    print(new_rest)
    p_rest = np.zeros((n, m))
    for i in range(k):
        p_rest[i] = instance.P[partial_solution[i]]
    p_rest[k:] = new_rest
    instance2 = FmCmax.FmCmax_Instance(name='',n=n,m=m,P=p_rest)
    solution = FlowShop.FlowShopSolution(instance2,job_schedule=[ps.Problem.Job(job_id,0,0) for job_id in l])
    solution.compute_objective()
    lb = solution.objective_value
    return lb

### 2- para_bound

In [6]:
def para_bound(instance, partial_solution):
    """
    paralelle AFTER CURRENT ONE
    """
    n, m = instance.P.shape
    k = len(partial_solution)
    
    # create a new array with the jobs in the partial solution
    p_partial = np.zeros((k, m))
    for i in range(k):
        p_partial[i] = instance.P[partial_solution[i]]
    
    p_rest = np.zeros((n, m))
    for i in range(n):
        if i not in partial_solution:
            p_rest[i] = instance.P[i]
        else:
            p_rest[i] = np.zeros((1,m))
    G = np.array(instance.P).transpose()
    solution = FlowShop.FlowShopSolution(instance,job_schedule=[ps.Problem.Job(job_id,0,0) for job_id in partial_solution])
    solution.compute_objective()
    lb = solution.objective_value 
    for machine in range(m):
        lb += np.max(p_rest[:,machine])
    return lb

### 3- last machine makespan bound

In [11]:
def last_machine_makespan(instance, partial_solution):
    P = instance.P
    n, m = P.shape
   
    #get the makespan of the partial solution ( completion time on the last machine )
    solution = FlowShop.FlowShopSolution(instance,job_schedule=[ps.Problem.Job(job_id,0,0) for job_id in partial_solution])
    solution.compute_objective()
    lb = solution.objective_value 

    #add the shortest remaining time of all the jobs on the last machine 
    remaining_jobs = [ i for i in range(n) if not i in partial_solution ]
    min_completion = P[remaining_jobs,m-1].sum()
    return lb + min_completion

### 4- lb2 bound

In [58]:
def lb2_bound(instance, sigma1):
    P = instance.P.T
    m = P.shape[0]  # number of machines
    n = P.shape[1]  # number of jobs
    c = np.zeros((m, n))  # completion times matrix
    u, v = find_bottleneck_machines(P, sigma1)  # find bottleneck machines
    
    max = 0
    lb = sum(P[0][i] for i in sigma1) 

    sum_max = 0
    for i in range(m):
        max = 0
        if i != u and i!=v:
            for j in range(n):
                if not j in sigma1:
                    if max < P[i][j]:
                        max = P[i][j]
            sum_max = sum_max + max
  
    # calculate lower bound on makespan
    
    two_machine_problem = [list(P[u][np.setdiff1d(np.arange(n), sigma1)]), list(P[v][np.setdiff1d(np.arange(n), sigma1)])]
    two_machine_data = np.array(two_machine_problem).T
    two_machine_seq = johnson_method(two_machine_data)
    two_machine_instance =  FmCmax.FmCmax_Instance(name='',n=len(two_machine_seq),m=2,P=two_machine_data)
    solution = FlowShop.FlowShopSolution(two_machine_instance,job_schedule=[ps.Problem.Job(job_id,0,0) for job_id in two_machine_seq])
    solution.compute_objective()
    two_machine_mk = solution.objective_value
#     print("Two machines makespan",two_machine_mk)
    lb = lb + two_machine_mk

    return lb

def johnson_method(processing_times):
    jobs, machines = processing_times.shape
    #print(jobs,machines)
    copy_processing_times = processing_times.copy()
    maximum = processing_times.max() + 1
    m1 = []
    m2 = []
    
    if machines != 2:
        raise Exception("Johson method only works with two machines")
        
    for i in range(jobs):
        minimum = copy_processing_times.min()
        position = np.where(copy_processing_times == minimum)
        
        if position[1][0] == 0:
            m1.append(position[0][0])
        else:
            m2.insert(0, position[0][0])
        
        copy_processing_times[position[0][0]] = maximum
        # Delete the job appended
    return m1+m2

def find_bottleneck_machines(P, sigma1):
    m = P.shape[0]  # number of machines
    n = P.shape[1]  # number of jobs
    # print(n,m)
    u, v = 0, 0
    max_idle_time = -1
    for i in range(m-1):
        for j in range(n):
            if j not in sigma1:
                idle_time = P[i][j] + P[i+1][j]
                if idle_time > max_idle_time:
                    max_idle_time = idle_time
                    u, v = i, i+1
    return u, v

### 5- Taillard

In [ ]:
def taillard(sequence, processing_times):
    print("lb")
    nb_jobs, nb_machines = processing_times.shape
    total_completion = processing_times[sequence,:].sum(axis=0)
    lower_bound = 0
    for i in range(nb_machines):
        b = []
        a = []
        for j in sequence:
            b.append(processing_times[j][:i].sum())
            a.append(processing_times[j][i+1:].sum())
        l = min(a) + min(b) + total_completion[i]
        if (l > lower_bound):
            lower_bound = l
    return lower_bound

## Tests

### Lecture des instances de taillard

In [7]:
with open("Taillard.pkl", "rb") as f:
    taillard = pickle.load(f)

### Choix de l'instance + method de lower bound

In [68]:
# les methods :
method1="last_machine-makespan"
method2="minseq_bound"
method3="para_bound"
method4="lb2_bound"
method5="taillard"
# faite votre choix:
instance_id=3 # de 0 a 9
choix=method4

### Creation de l'instance + objet branch&bound fsp

In [69]:
num_jobs=taillard[instance_id]["nj"]
num_machines=taillard[instance_id]["nm"]
P=np.array(taillard[instance_id]["P"]).transpose()
upper_bound=taillard[instance_id]["ub"]
lower_bound=taillard[instance_id]["lb"]
global l
l = [x for x in range(instance.n)]
# creation de l'instance
instance = FmCmax.FmCmax_Instance(name='',n=20,m=5,P=P)
stop_flag = False
start_time = time.time()
bb=FS_Branch_Bound(instance)
bb.choice=choix

### Execution (pandant 10min)

In [70]:
bb.solve()
print("taillard lower bound :",lower_bound)
print("taillard upper bound :",upper_bound)
print("time: ",bb.runtime,"s")
print(bb.best_solution)

taillard lower bound : 1268
taillard upper bound : 1293
time:  675.2047948999998 s
Objective : 1485
Jobs sequence : Job(id=15, start_time=0, end_time=259)	Job(id=6, start_time=29, end_time=366)	Job(id=0, start_time=63, end_time=449)	Job(id=9, start_time=116, end_time=556)	Job(id=12, start_time=181, end_time=641)	Job(id=10, start_time=190, end_time=731)	Job(id=16, start_time=229, end_time=812)	Job(id=17, start_time=265, end_time=830)	Job(id=7, start_time=313, end_time=857)	Job(id=13, start_time=385, end_time=893)	Job(id=14, start_time=411, end_time=962)	Job(id=4, start_time=483, end_time=1028)	Job(id=8, start_time=571, end_time=1110)	Job(id=18, start_time=613, end_time=1191)	Job(id=11, start_time=670, end_time=1254)	Job(id=3, start_time=749, end_time=1300)	Job(id=19, start_time=811, end_time=1372)	Job(id=1, start_time=906, end_time=1410)	Job(id=2, start_time=925, end_time=1454)	Job(id=5, start_time=1024, end_time=1485)
Machine_ID | Job_schedule (job_id , start_time , completion_time) | 

### Affichage de Gantt

In [ ]:
data = {
    1: [
        {'id':job.id , 'start': job.start_time, 'end': job.end_time} for job in bb.best_solution.machines[0].job_schedule 
    ],
    2: [
        {'id':job.id , 'start': job.start_time, 'end': job.end_time} for job in bb.best_solution.machines[1].job_schedule 

    ],
    3: [
        {'id':job.id , 'start': job.start_time, 'end': job.end_time} for job in bb.best_solution.machines[2].job_schedule 

    ],
        4: [
        {'id':job.id , 'start': job.start_time, 'end': job.end_time} for job in bb.best_solution.machines[3].job_schedule 

    ],
        5: [
        {'id':job.id , 'start': job.start_time, 'end': job.end_time} for job in bb.best_solution.machines[4].job_schedule 

    ],
}
# Sort the jobs in each machine by their start time
for machine in data:
    data[machine] = sorted(data[machine], key=lambda x: x['start'])

# Set up the plot
fig, ax = plt.subplots(figsize=(8, 4))

# Add each job as a horizontal bar with a unique color
colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11','C12']
for machine, jobs in data.items():
    for i, job in enumerate(jobs):
        ax.barh(machine, job['end'] - job['start'], left=job['start'], color=colors[i % len(colors)], align='center')
        ax.text(job['start'] + (job['end'] - job['start']) / 2, machine, '', ha='center', va='center', color='white')
        
# Set the title and axis labels
ax.set_title('Gantt Chart')
ax.set_xlabel('Time')
ax.set_ylabel('Machine')

# Set the ticks and labels for the y-axis
ax.set_yticks(list(data.keys()))
ax.set_yticklabels([f'Machine {machine}' for machine in data.keys()])

# Set the x-axis limits and ticks
last_time = max([job['end'] for jobs in data.values() for job in jobs])
ax.set_xlim(0, last_time)
ax.set_xticks(list(range(0, last_time + 1)))

# Add a label for the last time
ax.text(last_time + 0.5, 0.5, f'Last Time: {last_time}', va='center')

# Show the plot
plt.show()

## Autres method de lower bound proposee qui ne sont pas verifie d'etre des method exact

### 1- Machine based lower bound

In [ ]:
def machine_based_lower_bound(s, p, w):
    n, m = p.shape
    
    # Initialize variables
    L = np.zeros(m)
    for i in range(m):
        L[i]=p[s[len(s)-1],i]
    R = np.zeros((n+1, m))
    E = np.zeros((n+1, m))
    q=np.zeros((n,m,m))
    for j in range(n):
        for u in range(m):
            for v in range(m):
                if(u>-1 and v>u-1 and v<m):
                    q[j,u,v]=sum([p[j,r] for r in range(u,v+1)])
                else:
                    q[j,u,v]=0
                           
    E[len(s), 0] = L[0] 
    R[len(s)-1, 0] = L[0]
    for j in range(len(s), n):
        R[j, 0] = R[j-1, 0] + p[j-1, 0]
    for j in range(len(s)+1, n):
        E[j, 0] = R[j-1, 0]
    
    # Compute lower bounds for each machine
    for k in range(1, m):
        E[len(s), k] = max(L[k],max(E[len(s),r]+q[len(s),r,k-1] for r in range(0,k)))
        R[len(s)-1, k] = E[len(s), k]
        for t in range(len(s),n):
            R[t,k]=R[t-1,k]+p[t,k]
        for t in range(len(s)+1,n):
            E[t,k]=max(R[t-1,k],R[t,k-1])    
    # Compute the lower bound for the total flow time
    LB_k=np.zeros(m)
    U=[]
    for i in range(n):
        if(i not in s):
            U.append(i)
    for k in range(m):
        LB_k[k]=sum(E[t,k] for t in range(len(s),n))+sum(q[j,k,m-1] for j in U)+max(p[:,k])
    LB=max(LB_k[k] for k in range(m))/m
    return LB

### 2- Palmer bound

In [ ]:
def palmer_bound(process_times):
    """
    Compute the Palmer bound for the flow shop permutation problem.

    Args:
        process_times (list of lists): A list of lists representing the processing times for each job on each machine.
                                      The i-th element of the list represents the processing times for the i-th job,
                                      and the j-th element of the i-th element represents the processing time for the
                                      i-th job on the j-th machine.

    Returns:
        int: The Palmer bound for the given instance of the flow shop permutation problem.
    """
    num_jobs = len(process_times)
    num_machines = len(process_times[0])
    machine_sum = [0] * num_machines
    for i in range(num_jobs):
        for j in range(num_machines):
            machine_sum[j] += process_times[i][j]
            if j > 0:
                machine_sum[j] = max(machine_sum[j], machine_sum[j-1])
    return machine_sum[-1]

### 3- nehemiahs bound

In [29]:
def calculate_upper_bound(jobs):
    """
    Calculates an upper bound on the makespan of a given set of jobs using Johnson's rule.
    
    Args:
        jobs (List[List[int]]): A list of jobs, where each job is represented
            as a list of two processing times.
    
    Returns:
        int: An upper bound on the makespan of the given set of jobs.
    """
    n = len(jobs)
    # Sort the jobs by their "Johnson's rule" processing time
    sorted_jobs = sorted(jobs, key=lambda x: min(x))
    # Initialize the start times of each job on each machine
    machine_times = [[0, 0] for i in range(2)]
    # For each job in the sorted sequence
    for i in range(n):
        job = sorted_jobs[i]
        # Determine the machine with the earliest available time
        if machine_times[0][1] <= machine_times[1][1]:
            machine = 0
        else:
            machine = 1
        # Calculate the start time of the job on the machine
        start_time = machine_times[machine][1]
        # Update the machine times for the job
        machine_times[machine][0] = start_time
        machine_times[machine][1] = start_time + job[machine]
    # Return the maximum of the machine times
    return max(machine_times[0][1], machine_times[1][1])

def calculate_makespan(seq,jobs):
    """
    Calculates the makespan of a given sequence of jobs on m machines.
    
    Args:
        jobs (List[List[int]]): A list of jobs, where each job is represented
            as a list of m processing times.
        seq (List[int]): A sequence of job indices.
    
    Returns:
        int: The makespan of the given sequence of jobs.
    """
    n = len(jobs)
    m = len(jobs[0])
    # Initialize the finish times of each machine
    finish_times = [0] * m
    # Initialize the start times of each job
    start_times = [[0] * m for i in range(n)]
    # For each job in the sequence
    for i in range(n):
        job_index = seq[i]
        # For each machine
        for j in range(m):
            # Calculate the start time of the job on the machine
            if j == 0:
                start_times[job_index][j] = finish_times[j]
            else:
                start_times[job_index][j] = max(finish_times[j], start_times[job_index][j-1])
            # Calculate the finish time of the job on the machine
            finish_times[j] = start_times[job_index][j] + jobs[job_index][j]
    # Return the finish time of the last job on the last machine
    return finish_times[m-1]

def johnsons_rule(jobs, m):
    """
    Applies Johnson's rule to obtain a sequence of jobs for a flow shop
    scheduling problem.
    
    Args:
        jobs (List[List[int]]): A list of jobs, where each job is represented
            as a list of m processing times.
        m (int): The number of machines.
    
    Returns:
        List[int]: A sequence of job indices.
    """
    n = len(jobs)
    # Step 1: Create two lists of tuples containing job indices and their sum of processing times
    # on machine 1 and machine m, respectively
    list1 = [(i, sum(jobs[i][0:m])) for i in range(n)]
    list2 = [(i, sum(jobs[i][m-1::-1])) for i in range(n)]
    # Step 2: Sort list1 in increasing order of the sum of processing times
    list1.sort(key=lambda x: x[1])
    # Step 3: Sort list2 in increasing order of the sum of processing times
    list2.sort(key=lambda x: x[1])
    # Step 4: Initialize the output sequence
    seq = []
    # Step 5: While both list1 and list2 are not empty
    while list1 and list2:
        # If the smallest sum in list1 is smaller than the smallest sum in list2
        if list1[0][1] < list2[0][1]:
            # Append the job index with the smallest sum in list1 to the sequence
            seq.append(list1[0][0])
            # Remove the job index with the smallest sum in list1 from list1
            list1.pop(0)
        # Otherwise
        else:
            # Append the job index with the smallest sum in list2 to the sequence
            seq.append(list2[0][0])
            # Remove the job index with the smallest sum in list2 from list2
            list2.pop(0)
    # Step 6: Append the remaining job indices in list1 and list2 to the sequence
    for i in range(len(list1)):
        seq.append(list1[i][0])
    for i in range(len(list2)):
        seq.append(list2[i][0])
    return seq

def nehemiahs_bound(jobs, m):
    """
    Calculates a lower bound on the makespan of a flow shop scheduling problem
    using Nehemiah's bound.
    
    Args:
        jobs (List[List[int]]): A list of jobs, where each job is represented
            as a list of m processing times.
        m (int): The number of machines.
    
    Returns:
        int: The lower bound on the makespan.
    """
    n = len(jobs)
    # Apply Johnson's rule to obtain a lower bound
    seq = johnsons_rule(jobs, m)
    lb = calculate_makespan(seq, jobs)
    # Try all possible starting and ending jobs
    ub = float('inf')
    for i in range(n):
        for j in range(i+1, n):
            # Construct a modified list of jobs with i and j as the first and last jobs
            mod_jobs = [jobs[i]] + [jobs[k] for k in range(n) if k != i and k != j] + [jobs[j]]
            # Apply Johnson's rule to the modified list of jobs to obtain a lower bound
            seq = johnsons_rule(mod_jobs, m)
            lb = calculate_makespan(seq, mod_jobs)
            if lb < ub:
                ub = lb
    # Subtract the Nehemiah's bound from a known or calculated upper bound to get a lower bound
    # In this example, we assume that we have a function "calculate_upper_bound" that can calculate an upper bound
    upper_bound = calculate_upper_bound(jobs)  # Replace with your own function
    return upper_bound - ub

## Initialisation avec NEH (execute this cell then get back to the top and reexcute)

In [42]:
def neh_upper_bound(jobs, m):
    n = len(jobs)
    p = [[0]*m for i in range(n)]
    for i in range(n):
        for j in range(m):
            p[i][j] = jobs[i][j]

    # Step 1: Compute the processing time of each job
    processing_time = [sum(p[i]) for i in range(n)]

    # Step 2: Sort the jobs in decreasing order of processing time
    sorted_jobs = sorted(range(n), key=lambda i: processing_time[i], reverse=True)

    # Step 3: Initialize the schedule with the first job
    schedule = [sorted_jobs[0]]

    # Step 4: Insert each subsequent job into the schedule in a position that minimizes the makespan
    for i in range(1, n):
        best_pos = -1
        best_makespan = float('inf')
        for j in range(len(schedule)+1):
            temp_schedule = schedule[:j] + [sorted_jobs[i]] + schedule[j:]
            temp_makespan = compute_makespan(temp_schedule, p, m)
            if temp_makespan < best_makespan:
                best_makespan = temp_makespan
                best_pos = j
        schedule.insert(best_pos, sorted_jobs[i])

    return compute_makespan(schedule, p, m)

def compute_makespan(schedule, p, m):
    n = len(schedule)
    c = [[0]*m for i in range(n)]
    for i in range(n):
        for j in range(m):
            if i == 0 and j == 0:
                c[i][j] = p[schedule[i]][j]
            elif i == 0:
                c[i][j] = c[i][j-1] + p[schedule[i]][j]
            elif j == 0:
                c[i][j] = c[i-1][j] + p[schedule[i]][j]
            else:
                c[i][j] = max(c[i][j-1], c[i-1][j]) + p[schedule[i]][j]
    return c[n-1][m-1]
M=neh_upper_bound(instance.P, instance.m)
bb.M=M

## Taillard preparation (no need since we already have the file)

In [66]:
Ts=["54 83 15 71 77 36 53 38 27 87 76 91 14 29 12 77 32 87 68 94 79 3 11 99 56 70 99 60 5 56 3 61 73 75 47 14 21 86 5 77 16 89 49 15 89 45 60 23 57 64 7 1 63 41 63 47 26 75 77 40 66 58 31 68 78 91 13 59 49 85 85 9 39 41 56 40 54 77 51 31 58 56 20 85 53 35 53 41 69 13 86 72 8 49 47 87 58 18 68 28"
,"26 38 27 88 95 55 54 63 23 45 86 43 43 40 37 54 35 59 43 50 59 62 44 10 23 64 47 68 54 9 30 31 92 7 14 95 76 82 91 37 78 90 64 49 47 20 61 93 36 47 70 54 87 13 40 34 55 13 11 5 88 54 47 83 84 9 30 11 92 63 62 75 48 23 85 23 4 31 13 98 69 30 61 35 53 98 94 33 77 31 54 71 78  9 79 51 76 56 80 72"
,"77 94  9 57 29 79 55 73 65 86 25 39 76 24 38 5 91 29 22 27 39 31 46 18 93 58 85 58 97 10 79 93 2 87 17 18 10 50 8 26 14 21 15 10 85 46 42 18 36 2 44 89 6 3 1 43 81 57 76 59 11 2 36 30 89 10 88 22 31 9 43 91 26 3 75 99 63 83 70 84 83 13 84 46 20 33 74 42 33 71 32 48 42 99 7 54 8 73 30 75"
,"53 19 99 62 88 93 34 72 42 65 39 79  9 26 72 29 36 48 57 95 93 79 88 77 94 39 74 46 17 30 62 77 43 98 48 14 45 25 98 30 90 92 35 13 75 55 80 67  3 93 54 67 25 77 38 98 96 20 15 36 65 97 27 25 61 24 97 61 75 92 73 21 29  3 96 51 26 44 56 31 64 38 44 46 66 31 48 27 82 51 90 63 85 36 69 67 81 18 81 72"
,"61 86 16 42 14 92 67 77 46 41 78  3 72 95 53 59 34 66 42 63 27 92  8 65 34  6 42 39  2  7 85 32 14 74 59 95 48 37 59  4 42 93 32 30 16 95 58 12 95 21 74 38  4 31 62 39 97 57  9 54 13 47  6 70 19 97 41  1 57 60 62 14 90 76 12 89 37 35 91 69 55 48 56 84 22 51 43 50 62 61 10 87 99 40 91 64 62 53 33 16"
," 71 27 55 90 11 18 42 64 73 95 22 53 32  5 94 12 41 85 75 38 13 11 73 43 27 33 57 42 71  3 11 49  8  3 47 58 23 79 99 23 61 25 52 72 89 75 60 28 94 95 18 73 40 61 68 75 37 13 65  7 21  8  5  8 58 59 85 35 84 97 93 60 99 29 94 41 51 87 97 11 91 13  7 95 20 69 45 44 29 32 94 84 60 49 49 65 85 52  8 58"
," 15 64 64 48  9 91 27 34 42  3 11 54 27 30  9 15 88 55 50 57 28  4 43 93  1 81 77 69 52 28 28 77 42 53 46 49 15 43 65 41 77 36 57 15 81 82 98 97 12 35 84 70 27 37 59 42 57 16 11 34 1 59 95 49 90 78  3 69 99 41 73 28 99 13 59 47  8 92 87 62 45 73 59 63 54 98 39 75 33  8 86 41 41 22 43 34 80 16 37 94"
," 34 20 57 47 62 40 74 94  9 62 86 13 78 46 83 52 13 70 40 60 5 48 80 43 34  2 87 68 28 84 30 35 42 39 85 34 36  9 96 84 86 35  5 93 74 12 40 95 80  6 92 14 83 49 36 38 43 89 94 33 28 39 55 21 25 88 59 40 90 18 33 10 59 92 15 77 31 85 85 99 8 91 45 55 75 18 59 86 45 89 11 54 38 41 64 98 83 36 61 19"
," 37 36  1  4 64 74 32 67 73  7 78 64 98 60 89 49  2 79 79 53 59 16 90  3 76 74 22 30 89 61 39 15 69 57  9 13 71  2 34 49 65 94 96 47 35 34 84  3 60 34 70 57  8 74 13 37 87 71 89 57 70  3 43 14 26 83 26 65 47 94 75 30  1 71 46 87 78 76 75 55 94 98 63 83 19 79 54 78 29  8 38 97 61 10 37 16 78 96  9 91"
," 27 92 75 94 18 41 37 58 56 20  2 39 91 81 33 14 88 22 36 65 79 23 66  5 15 51  2 81 12 40 59 32 16 87 78 41 43 94  1 93 22 93 62 53 30 34 27 30 54 77 24 47 39 66 41 46 24 23 68 50 93 22 64 81 94 97 54 82 11 91 23 32 26 22 12 23 34 87 59  2 38 84 62 10 11 93 57 81 10 40 62 49 90 34 11 81 51 21 39 27"
]
Taillard = [
    {
        "ub": 1278,
        "lb": 1232,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    },
        {
        "ub": 1359,
        "lb": 1290,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    },
        {
        "ub": 1081,
        "lb": 1073,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    },
        {
        "ub": 1293,
        "lb": 1268,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    },
        {
        "ub": 1236,
        "lb": 1198,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    },
        {
        "ub": 1195,
        "lb": 1180,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    },
        {
        "ub": 1239,
        "lb": 1226,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    },
        {
        "ub": 1206,
        "lb": 1170,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    },
            {
        "ub": 1230,
        "lb": 1206,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    },
            {
        "ub": 1108,
        "lb": 1082,
        "nj": 5,
        "nm": 20,
        "P": [[] for _ in range(5)],
    }
]
len(Ts)

for i in range(len(Ts)):
    Ts[i] = Ts[i].strip()  # removes spaces from beginning and end
    Ts[i] = " ".join(Ts[i].split()) 

for idx in range(len(Ts)):
    T=Ts[idx].split(" ")
    for idx2,value in enumerate(T):
        Taillard[idx]["P"][int(idx2/Taillard[idx]["nm"])].append(int(value))
        
# Save the data to a file
with open("Taillard.pkl", "wb") as f:
    pickle.dump(Taillard, f)